In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
!pip install pymorphy2
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

---

## Семантический анализ твитов

Сегодня мы построим классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой.
Загрузим данные для анализа

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')   # чтобы можно было оттуда считать файл с данными для этого задания

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/data/tweets_example.xlsx')
df.loc[16:25]

---

Все колонки таблицы могут содержать информацию о тональности твита, но мы будем ориентироваться исключительно на текст и на столбец отнесения к классу positiv.
Заменим значение -1 в колонке positive на 0

In [ ]:
df.positive[df.positive==-1]=0
df.loc[16:25]

-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

In [ ]:
df_pos = pd.read_csv('/content/drive/MyDrive/data/positive.csv', sep=';', names=['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'])
df_neg = pd.read_csv('/content/drive/MyDrive/data/negative.csv', sep=';', names=['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'])
df_neg.positive[df_neg.positive==-1]=0

df_all = pd.concat([df_pos, df_neg], ignore_index=True)
df_all.id.duplicated()

print ('All tweets count : ', df_all['positive'].count())

count = (df_all['positive'] == 1).sum()
print('Positive tweets count : ', count)

count_neg = (df_all['positive'] == 0).sum()
print('Negative tweets count : ', count_neg)

df_all['text'] = df_all['text'].str.replace('ё','е')
df_all['text'] = df_all['text'].str.replace('Ё','Е')
df_all[7:8]

### Очистка и предобработка данных

Перед разработкой классификатора нам необходимо очистить и предобработать данные.

Начнем с очистки данных

----------------------------

***Внимание!*** Библиотека [*nltk*](https://www.nltk.org) может содержать не все компоненты. В случае возникновения ошибки необходимо запустить скрипт

*import nltk   
nltk.download()*

В открывшемся окне необходимо выбрать и установить требуемые компоненты

----------------------------

Приведем весь текст к строчным буквам:

In [ ]:
df.text = df.text.str.lower()
df.text.loc[19:22]

# import nltk
# nltk.download()

---

Оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей:

In [ ]:
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

---

Мы анализируем русскоязычный твиттер, поэтому английские слова, а так же числа, будут представлять частные случаи и формировать шум в данных. Но могут возникнуть задачи, где удаляемые слова и числа важны. В этом случае потребуется более взвешенный подход к очистке. Вам могут помочь [константы модуля *string*](https://docs.python.org/3/library/string.html)

Разобьем тексты на слова с помощью *word_tokenize*:

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
df.text = list(map(word_tokenize, df.text))
df.text.loc[19:22]

---

В каждом языке имеются так называемые стоп-слова - это, например, предлоги, союзы, местоимения и т.д. Стоп-слова не несут смысловой нагрузки, но при этом встречаются достаточно часто. Существует множество словарей стоп-слов, мы воспользуемся словарем библиотеки *nltk*. При решении конкретных задач вы можете как расширить словарь стоп-слов, так и удалить из него любые слова.

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

---

Удалим стоп-слова из наших данных

In [ ]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]
# df_test.text.loc[0:15]

---

Проведем [лемматизацию](https://ru.wikipedia.org/wiki/Лемматизация) полученных слов

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

---

Теперь необходимо удалить все слова, которые встречаются только 1 раз

In [ ]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text.loc[19:22]

---

На этом очистка данных завершена. Можно ли утверждать, что очистка идеальна? Однозначно нет! Но, ее может оказаться достаточно для решения нашей задачи.   
Какие еще задачи могут возникнуть при очистке текстовых данных? Вот далеко неполный список:
- Обработка больших документов и больших коллекций текстовых документов, которые не помещаются в память.
- Извлечение текста из разметки, такой как HTML, PDF или другие структурированные форматы документов.
- Транслитерация символов с других языков.
- Декодирование символов Юникода в нормализованную форму, такую как UTF8
- Обработка доменных имен, фраз и сокращений.
- Обработка или удаление чисел, таких как даты и суммы.
- Поиск и исправление распространенных опечаток и ошибок в написании.

Можно очень долго заниматься очисткой и не достичь идеального результата. Лучше подойти к задаче итеративно - осуществить стандартную очистку и посмотреть на результат, если результат недостаточный, то провести дополнительные мероприятия по очистке.

In [ ]:
len(text_tokens), len(words_to_del), len(text_tokens) - len(words_to_del)

---

После очистки в наших данных осталось всего 62 слова (это не уникальные повторяющиеся слова, уникальных всего 29). Этого мало для построения классификаторов, но позволило существенно сократить время для знакомства с очисткой данных. В вашем проекте после очистки останется более 1,4 млн слов.


После очистки могут оказаться пустые твиты, т.е. эти твиты состояли из слов, записанных латиницей, стоп-слов, чисел, знаков припинания и уникальных слов. Такие твиты необходимо удалить из данных:

In [ ]:
len(df[df.text == '']), len(df[(df.text == '') & (df.positive == 1)])

In [ ]:
df = df.drop(df[df.text == ''].index, axis = 0)
len(df)

---

После удаления пустых твитов у нас осталось 33 записи

----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:   
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

In [ ]:
df_test = df_all
df_test[7:8]

In [ ]:
df_test.text = df_test.text.str.lower()
df_test.text.loc[7:7]

In [ ]:
import nltk
nltk.download('punkt')

df_test.text = df_test.text.str.replace(r"[^А-Яа-я]"," ")
df_test.text.loc[0:10]

In [ ]:
from nltk.tokenize import word_tokenize

df_test.text = list(map(word_tokenize, df_test.text))
df_test.text.loc[0:8]

In [ ]:
# import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
# russian_stopwords

In [ ]:
df_before = df_test
# df_test = df_before

In [ ]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df_test.text = list(map(delete_stopword, df_test.text))
df_test.text.loc[19:22]


In [ ]:
df_test.text.sum

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df_test.text = list(map(lemmatization, df_test.text))
df_test.text.loc[0:15]

Удаление всех слов, которые встречаются только 1 раз:

---



In [ ]:
df_del = df_test
df_del[0:15]

In [ ]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df_del.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df_del.text = list(map(delete_word, df_del.text))
df_del.text = list(map(to_str, df_del.text))
df_del.text.loc[0:8]

In [ ]:
print("Всего слов: ", len(text_tokens)) 
print("Удалено: ", len(words_to_del))
print("Осталось слов: ", len(text_tokens) - len(words_to_del))

Всего слов:  1477000
Удалено:  53547
Осталось слов:  1423453


In [ ]:
print("Пустых твитов: ", len(df_del[df_del.text == '']))
print("Из них позитивных: ", len(df_del[(df_del.text == '') & (df_del.positive == 1)])

Пустых твитов:  0
Из них позитивных:  0


In [ ]:
df_after_del = df_del
# df_del = df_after_del
df_del = df_del.drop(df_del[df_del.text == ''].index, axis = 0)
len(df_del)

225962

In [ ]:
df_del[0:15]

Для разработки моделей нам необходимо оцифровать полученные данные. Мы воспользуемся двумя методами: мешком слов [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и TF-IDF [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Но в начале необходимо разбить данные на обучающую и тестовую выборки.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.2, random_state=21)

---

Рассмотрим количество твитов в выборке для обучения, из них позитивных, и в выборке для теста, из них позитивных

In [ ]:
len(y_train), y_train.sum(), len(y_test), y_test.sum()

---

### Кодировка данных   

Кодируем наши данные мешком слов и tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

---

### Классификаторы

Построим классификатор с помощью логистической регрессии:   
на основе мешка слов:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.40      1.00      0.57         2

    accuracy                           0.57         7
   macro avg       0.70      0.70      0.57         7
weighted avg       0.83      0.57      0.57         7

train
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.92      0.92      0.92        13

    accuracy                           0.92        26
   macro avg       0.92      0.92      0.92        26
weighted avg       0.92      0.92      0.92        26



---

на основе tf-idf:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.40      1.00      0.57         2

    accuracy                           0.57         7
   macro avg       0.70      0.70      0.57         7
weighted avg       0.83      0.57      0.57         7

train
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       1.00      0.92      0.96        13

    accuracy                           0.96        26
   macro avg       0.96      0.96      0.96        26
weighted avg       0.96      0.96      0.96        26



---

Видно, что модели переобучены - это следствие малого количества данных.

---

Построим классификатор с помощью случайного леса [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):   
на основе мешка слов:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=3, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.50      0.67         2

    accuracy                           0.86         7
   macro avg       0.92      0.75      0.79         7
weighted avg       0.88      0.86      0.84         7

train
              precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       1.00      0.85      0.92        13

    accuracy                           0.92        26
   macro avg       0.93      0.92      0.92        26
weighted avg       0.93      0.92      0.92        26



---

на основе tf-idf:

In [ ]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         2

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

train
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.92      0.92      0.92        13

    accuracy                           0.92        26
   macro avg       0.92      0.92      0.92        26
weighted avg       0.92      0.92      0.92        26



---

Получили очень хороший результат. Надо проверить, получится ли так же хорошо на всем объеме данных.

---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

***Рекомендация:*** в чек-листе содержится объемный обучающий материал, поэтому лучше не затягивать с решением заданий

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_del.text, df_del.positive, test_size=0.3, random_state=21)

In [ ]:
X_train
# y_train

In [ ]:
print("Твитов в выборке для обучения: ", len(y_train))
print("Из них позитивных: ", y_train.sum())
print("Твитов в выборке для теста: ", len(y_test))
print("Из них позитивных: ", y_test.sum())

# Кодировка данных

---


Кодируем данные мешком слов:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

Кодируем данные tf-idf:

---



In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

# Классификатор с помощью случайного леса RandomForestClassifier:
на основе мешка слов:

---



In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))


на основе tf-idf:

---

In [ ]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))